In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
import joblib
import os

In [3]:
dx=pd.read_csv('/content/real_legitimate_v1.csv')

In [4]:
dy=pd.read_csv('/content/real_malware_v1.csv')

In [5]:
df = pd.concat([dx, dy], ignore_index=True)
df.to_csv('combined.csv', index=False)

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df=df.to_csv('shuffled_dataset.csv', index=False)

In [7]:
dr=pd.read_csv('/content/shuffled_dataset.csv')

In [8]:
data={
    'Package':dr['Package'][:5000],
    'Malware':dr['Malware'][:5000],
    'prctl':dr['prctl'][:5000],
    'sha256':dr['sha256'][:5000],
    'EarliestModDate':dr['EarliestModDate'][:5000],
    'Malware':dr['Malware'][:5000],
    'write': dr['write'][:5000],
    'read':dr['read'][:5000],
    'write':dr['write'][:5000],
    'munmap':dr['munmap'][:5000],
    'mprotect':dr['mprotect'][:5000],
    'madvise':dr['madvise'][:5000],
    'ioctl':dr['ioctl'][:5000],
    'writev':dr['writev'][:5000],
    'dup':dr['dup'][:5000],
    'lseek':dr['lseek'][:5000],
    'mmap2':dr['mmap2'][:5000],
    'fstat64':dr['fstat64'][:5000],
    'rt_sigprocmask':dr['rt_sigprocmask'][:5000],
   'recvfrom':dr['recvfrom'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'lseek':dr['lseek'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'futex':dr['futex'][:5000],
    'pread':dr['pread'][:5000],
     'FilesInsideAPK':dr['FilesInsideAPK'][:5000],
   'TimesSubmitted':dr['TimesSubmitted'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'Activities':dr['Activities'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'NrIntActivities':dr['NrIntActivities'][:5000],
    'NrIntActivitiesActions':dr['NrIntActivitiesActions'][:5000],
    'TotalIntentFilters':dr['TotalIntentFilters'][:5000],
    'Scanners':dr['Scanners'][:5000],
    'MalFamily':dr['MalFamily'][:5000]
    }
dg=pd.DataFrame(data)
dg

,Package,Malware,prctl,sha256,EarliestModDate,write,read,munmap,mprotect,madvise,...,futex,pread,FilesInsideAPK,TimesSubmitted,Activities,NrIntActivities,NrIntActivitiesActions,TotalIntentFilters,Scanners,MalFamily
0,com.mrselected.scholarship.fin\n,0,83,40f89565f035513cca24c7f92ba2dc0670399aff776774...,3/20/2011,9,14,53,593,41,...,106,72,23.0,6.0,2,1,1,1,66.0,NaN
1,com.JSanders.LabTimerSE\n,0,3,2f5ba7c631f1232aca7be9de8b3906a56e3e4fec2cdf46...,7/18/2010,4,2,0,2,2,...,4,0,67.0,7.0,4,1,1,1,66.0,NaN
2,com.android.security.widkdof8\n,1,228,71f9857e366b11181a28c846e783c88a427c509007422c...,9/5/2015,207,184,98,1406,84,...,425,100,9.0,6.0,2,2,2,5,65.0,Locker/SLocker Ransomware
3,com.yje.ujdepd\n,1,504,6e0495ee8a19a298b9a7416363727f59301af63441cfb0...,6/12/2017,475,416,293,2706,291,...,913,278,340.0,3.0,8,3,3,14,65.0,SMSreg
4,eu.borzaindustries.girls_memory\n,0,3,a19207b0529ad348ce7ab43a528cab2c13e99e42b16c7b...,11/24/2010,3,2,0,2,2,...,3,0,100.0,5.0,9,1,1,1,66.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,com.SKTPDF.MRstomachEng\n,0,5,f434fbed01865d0268c7ceeddca67805f5ecbd5b9f4d2e...,3/27/2011,2,2,28,4,4,...,3,56,323.0,4.0,13,1,1,1,65.0,NaN
4996,net.qiixiao.android.nosomking\n,1,21003,e37d6bc04156b75b8db19a6db8ce4473931ed953e94527...,6/29/2011,3,6301,40,14158,14008,...,7817,5,58.0,9.0,6,1,1,1,66.0,Gappusin
4997,gerald0921.game.basketball\n,1,130,7d8ef4c88f1e3e60ea9b426fb38f696aeaa6f9db6145f2...,11/22/2010,28,7,29,880,27,...,54,5,90.0,5.0,7,2,2,2,67.0,Airpush/StopSMS
4998,com.pearsestreet.Todo\n,0,0,8938019b38519221d7326c6e5c1b724e9f9bda262da9e0...,1/10/2011,1,2,1,0,0,...,0,2,31.0,10.0,1,1,1,1,63.0,NaN


In [9]:
dg.isnull().sum()

Package                      0
Malware                      0
prctl                        0
sha256                       1
EarliestModDate              1
write                        0
read                         0
munmap                       0
mprotect                     0
madvise                      0
ioctl                        0
writev                       0
dup                          0
lseek                        0
mmap2                        0
fstat64                      0
rt_sigprocmask               0
recvfrom                     0
getsockopt                   0
clock_gettime                0
futex                        0
pread                        0
FilesInsideAPK               1
TimesSubmitted               1
Activities                   1
NrIntActivities              1
NrIntActivitiesActions       1
TotalIntentFilters           1
Scanners                     1
MalFamily                 2203
dtype: int64

In [10]:
dg=dg.fillna(0)

In [11]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [12]:
dg.dtypes

Package                    object
Malware                     int64
prctl                       int64
sha256                     object
EarliestModDate            object
write                       int64
read                        int64
munmap                      int64
mprotect                    int64
madvise                     int64
ioctl                       int64
writev                      int64
dup                         int64
lseek                       int64
mmap2                       int64
fstat64                     int64
rt_sigprocmask              int64
recvfrom                    int64
getsockopt                  int64
clock_gettime               int64
futex                       int64
pread                       int64
FilesInsideAPK            float64
TimesSubmitted            float64
Activities                 object
NrIntActivities            object
NrIntActivitiesActions     object
TotalIntentFilters         object
Scanners                  float64
MalFamily     

In [13]:
dg.dtypes

Package                    object
Malware                     int64
prctl                       int64
sha256                     object
EarliestModDate            object
write                       int64
read                        int64
munmap                      int64
mprotect                    int64
madvise                     int64
ioctl                       int64
writev                      int64
dup                         int64
lseek                       int64
mmap2                       int64
fstat64                     int64
rt_sigprocmask              int64
recvfrom                    int64
getsockopt                  int64
clock_gettime               int64
futex                       int64
pread                       int64
FilesInsideAPK            float64
TimesSubmitted            float64
Activities                 object
NrIntActivities            object
NrIntActivitiesActions     object
TotalIntentFilters         object
Scanners                  float64
MalFamily     

In [14]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_values = label_encoder.fit_transform(dg['MalFamily'].astype(str))
encoded_values = encoded_values.reshape(-1, 1)
dg['MalFamily'] = encoded_values
dg['MalFamily'].max()
encoded=label_encoder.fit_transform(dg['Package'].astype(str))
dg['Package']=encoded

In [15]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [16]:
dg=dg.fillna(0)

In [17]:
import pandas as pd
dg['Activities'] = pd.to_numeric(dg['Activities'], errors='coerce')
dg['TotalIntentFilters']=pd.to_numeric(dg['TotalIntentFilters'],errors='coerce')
dg['NrIntActivities']=pd.to_numeric(dg['NrIntActivities'],errors='coerce')
dg['NrIntActivitiesActions']=pd.to_numeric(dg['NrIntActivitiesActions'],errors='coerce')
dg['sha256']=pd.to_numeric(dg['sha256'],errors='coerce')

In [18]:
if pd.isnull(dg['EarliestModDate']).any():
    # Handle or fill the missing values if necessary
    dg['EarliestModDate'].fillna('fill_value', inplace=True)  # Replace 'fill_value' with an appropriate value
# Convert the 'EarliestModDate' column to datetime
dg['EarliestModDate'] = pd.to_datetime(dg['EarliestModDate'], errors='coerce')

In [19]:
print(dg['EarliestModDate'].dtype)

datetime64[ns]


In [20]:
dg.isnull().sum()

Package                      0
Malware                      0
prctl                        0
sha256                    4999
EarliestModDate             92
write                        0
read                         0
munmap                       0
mprotect                     0
madvise                      0
ioctl                        0
writev                       0
dup                          0
lseek                        0
mmap2                        0
fstat64                      0
rt_sigprocmask               0
recvfrom                     0
getsockopt                   0
clock_gettime                0
futex                        0
pread                        0
FilesInsideAPK               0
TimesSubmitted               0
Activities                  47
NrIntActivities             47
NrIntActivitiesActions      47
TotalIntentFilters          47
Scanners                     0
MalFamily                    0
dtype: int64

In [21]:
dg=dg.fillna(0)

In [22]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [28]:
# %% [code]
#!/usr/bin/env python
# coding: utf-8

# # Experiment 3: Train in batches by month and test in the next month

# In[1]:


import pandas as pd
import numpy as np


# In[2]:


# initial year defines the training (< initial_year) and testing sets (>= initial_year)
initial_year = "2017-12-30"


# In[4]:


# In[6]:


# Transform submission_date in a datetime column:
dg['EarliestModDate'] =  pd.to_datetime(dg['EarliestModDate'],errors='coerce')


# In[7]:


# Group data by months and create subsets to train and test:
dataTrain =dg[dg['EarliestModDate'] <=initial_year]
dataTest = dg[dg['EarliestModDate'] >initial_year]
print("Train shape:", dataTrain.shape)
print("Test shape:", dataTest.shape)


# In[9]:

# In[11]:


# Normalization method:
def normalization(X_train, X_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    XX_train = scaler.transform(X_train)
    XX_test = scaler.transform(X_test)
    return(XX_train, XX_test)


# In[13]:


# Iterate over months in test data:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
#from sklearn.ensemble import RandomForestClassifier as Classifier
from sklearn.linear_model import SGDClassifier as Classifier
# group by year-month
dataTestTime = dataTest.groupby([dataTest.EarliestModDate.dt.year, dataTest.EarliestModDate.dt.month])
# get data train labels
y_train = np.array(dataTrain["Malware"])
# remove unused columns from dataTrain
UNUSED_COLUMNS = ["Malware", "sha256", "EarliestModDate"]
for c in UNUSED_COLUMNS:
    del dataTrain[c]
# get dataTrain values
X_dataTrain = dataTrain.values
# arrays to save information
accs = [] # accuracy
f1s = [] # f1score
recall = [] # recall
precision = [] # precision
years = []
months = []
# iterate over year and month
for (year, month), dataTestMonth in dataTestTime:
    # get data test labels
    y_test = np.array(dataTestMonth["Malware"])
    # remove unused columns from data test
    for c in UNUSED_COLUMNS:
        del dataTestMonth[c]
    # get dataTestMonth values
    X_dataTest = dataTestMonth.value
    # normalization
    XX_train, XX_test = normalization(X_dataTrain, X_dataTest)
    # classifier
    # clf = RandomForestClassifier(random_state=0)
    clf = Classifier(random_state=0)
    # train classifier
    clf.fit(XX_train,y_train)
    # predict test data
    y_pred = clf.predict(XX_test)
    # update train data and labels with actual month
    X_dataTrain = np.append(X_dataTrain, X_dataTest, axis=0)
    y_train = np.concatenate((y_train, np.array(y_test)), axis=0)
    print("{}-{}".format(year,month))
    # save metrics
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    re = recall_score(y_test, y_pred)
    pr = precision_score(y_test, y_pred)
    if (f1 !=0) and (re != 0) and (pr != 0):
        accs.append(acc)
        f1s.append(f1)
        recall.append(re)
        precision.append(pr)
        years.append(year)
        months.append(month)
        print(acc)



Train shape: (4745, 30)
Test shape: (163, 30)
2018-1
0.4375
2018-2
0.75
2018-3
0.8260869565217391
2018-4
0.3333333333333333
2018-5
0.6666666666666666
2018-6
0.8888888888888888
2018-7
2018-9
2018-11


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

2019-1
2019-5
1.0
2019-6
2019-8
2019-10
1.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-scor

2019-11
1.0
2019-12
2020-4
0.6
2020-5
1.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2020-11
1.0
2107-11


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
